In [ ]:
import random
import os 
import cv2

def readImageFile(file_path):

    # read image as an 8-bit array
    img_bgr = cv2.imread(file_path)

    # convert to RGB
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # convert the original image to grayscale
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    return img_rgb, img_gray


def saveImageFile(img_rgb, file_path):
    try:
        # convert BGR
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

        # save the image
        success = cv2.imwrite(file_path, img_bgr)
        if not success:
            print(f"Failed to save the image to {file_path}")
        return success

    except Exception as e:
        print(f"Error saving the image: {e}")
        return False

class ImageDataLoader:
    def __init__(self, directory, shuffle=False, transform=None):
        self.directory = directory
        self.shuffle = shuffle
        self.transform = transform

        # get a sorted list of all files in the directory
        self.file_list = [f for f in os.listdir(directory) if f.endswith(('.jpg', '.png', '.jpeg'))]
        self.file_list = sorted(self.file_list)

        if not self.file_list:
            raise ValueError("No image files found in the directory.")

        # shuffle file list if required
        if self.shuffle:
            random.shuffle(self.file_list)

        # get the total number of batches
        self.num_batches = len(self.file_list)

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        # fill in with your own code below
        for file_name in self.file_list:
            file_path = os.path.join(self.directory, file_name)

            # read the image
            img_rgb, img_gray = readImageFile(file_path)

            # apply transformation 
            if self.transform:
                img_rgb = self.transform(img_rgb)
                img_gray = self.transform(img_gray)

            yield img_rgb, img_gray, file_name  


image_loader = ImageDataLoader(directory="../data", shuffle=False)


for img_rgb, img_gray, filename in image_loader:
    print(f"Loaded image: {filename}, Shape: {img_rgb.shape}")

def removeHair(img_org, img_gray, kernel_size=25, threshold=10, radius=3):
    """detects and removes hair from images"""
    # kernel for the morphological filtering
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (kernel_size, kernel_size))

    # perform the blackHat filtering on the grayscale image to find the hair countours
    blackhat = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)

    # intensify the hair countours in preparation for the inpainting algorithm
    _, thresh = cv2.threshold(blackhat, threshold, 255, cv2.THRESH_BINARY)

    # inpaint the original image depending on the mask
    img_out = cv2.inpaint(img_org, thresh, radius, cv2.INPAINT_TELEA)

    return blackhat, thresh, img_out


Loaded image: image0.jpg, Shape: (450, 600, 3)
Loaded image: img_0410.png, Shape: (288, 384, 3)
Loaded image: img_0414.png, Shape: (384, 384, 3)
Loaded image: img_0423.png, Shape: (288, 384, 3)
Loaded image: img_0517.png, Shape: (288, 384, 3)
Loaded image: img_0569.png, Shape: (288, 384, 3)


### Convert to grayscale and save in folder

In [ ]:
# initializing folders for results
input_dir = "/Users/philiphansen/Documents/itu/projects_in_ds/group-D-images"
output_dir = os.path.join(input_dir, "processed_images")
gray_output_dir = os.path.join(output_dir, "gray_images")

os.makedirs(gray_output_dir, exist_ok=True)

# load all the images using the data loader
group_D_images = ImageDataLoader(directory=input_dir, shuffle=False)

# Save the grayscale images to the gray output folder
i = 393
for img_rgb, img_gray, filename in group_D_images:
    output_path = os.path.join(gray_output_dir, f"group_D_img_gray_{i}.png")
    cv2.imwrite(output_path, img_gray)
    i += 1

print(f"Now all grayscale images has been saved in the folder {gray_output_dir}")

Now all grayscale images has been saved in the folder /Users/philiphansen/Documents/itu/projects_in_ds/group-D-images/processed_images/gray_images


In [44]:
group_D_file_numbers = list(range(393, 593))

for i in range(len(group_D_images)):
    # read all the images
    img_rgb, img_gray = readImageFile(f"/Users/philiphansen/Documents/itu/projects_in_ds/group-D-images/{group_D_images.file_list[i]}")
    filename = os.path.splitext(os.path.basename(group_D_images.file_list[i]))[0]

    # don't proces images not from group D
    if int(filename.removeprefix("img_0")) not in group_D_file_numbers:
        continue

    #apply the hair removal
    blackhat, thresh, img_out = removeHair(img_rgb, img_gray, kernel_size=25, threshold=2, radius=3)

    # save the processed images
    cv2.imwrite(os.path.join(output_dir, f"{filename}_blackhat.png"), blackhat)
    cv2.imwrite(os.path.join(output_dir, f"{filename}_thresh.png"), thresh)
    cv2.imwrite(os.path.join(output_dir, f"{filename}_processed.png"), cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR))

print(f"All processed images have been saved in {output_dir}")

All processed images have been saved in /Users/philiphansen/Documents/itu/projects_in_ds/group-D-images/processed_images
